# Create a Spark Session

In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.functions import max, min, avg, col, count, date_format

# Create a SparkSession
spark = SparkSession.builder.appName("UberDataAnalysis").getOrCreate()

# Load the dataset into a DataFrame
uber_df = spark.read.csv("uber.csv", header=True, inferSchema=True)
uber_df.show(5)  # Display the first 5 rows of the DataFrame

+---------+-----------+--------+------+---------------+--------+--------------+
|     Date|Time(Local)|Eyeballs|Zeroes|Completed_Trips|Requests|Unique_Drivers|
+---------+-----------+--------+------+---------------+--------+--------------+
|10-Sep-12|          7|       5|     0|              2|       2|             9|
|10-Sep-12|          8|       6|     0|              2|       2|            14|
|10-Sep-12|          9|       8|     3|              0|       0|            14|
|10-Sep-12|         10|       9|     2|              0|       1|            14|
|10-Sep-12|         11|      11|     1|              4|       4|            11|
+---------+-----------+--------+------+---------------+--------+--------------+
only showing top 5 rows



# Which date had most completed trips during 3 week period

In [9]:
completed_trips=uber_df.groupBy("Date").sum("Completed_Trips")
completed_trips.show()

most_completed_trips=completed_trips.orderBy("sum(Completed_Trips)",ascending=False).select("Date").first()["Date"]
print(most_completed_trips)

+---------+--------------------+
|     Date|sum(Completed_Trips)|
+---------+--------------------+
|11-Sep-12|                  40|
|13-Sep-12|                  47|
|18-Sep-12|                  42|
|21-Sep-12|                 190|
|12-Sep-12|                  91|
|19-Sep-12|                  41|
|14-Sep-12|                 305|
|16-Sep-12|                 150|
|23-Sep-12|                 111|
|24-Sep-12|                   4|
|22-Sep-12|                 248|
|20-Sep-12|                  70|
|10-Sep-12|                  26|
+---------+--------------------+

14-Sep-12


# What was the highest number of completed trips within a 24-hour period?

In [12]:
#Group the data by 24-hour window
completed_trips_by_window=uber_df.groupBy(window("Time(Local)","24 hours"))#2.agg(sum("Completed_Trips").alias("total_completed_trips")).orderBy("total_completed_trips",ascending=False)
completed_trips_by_window.show()

AttributeError: 'GroupedData' object has no attribute 'show'